# Building Text Classifiers

Frank Neugebauer
October 1, 2019

The objective of this project is to demonstrate the accuracy of different text classifiers in Python. To get that output, corpora from Reddit that show categorized and controversial entries is used.

Some of what's demonstrated:
* Reading JSON files
* Sampling to increase performance
* Tokenization
* Creating vectors as features
* Logistic regression with different penalities
* Multinomrial Naive Bayes

First, import everything that's needed.

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Loading and Sampling Data

The next step is to load the data, but it's massive and in order to avoid processing problems, I loaded the full data outside of GitHub and then took a sample from each. Then, that sample data is saved as separate CSV files.

Note that the controversial data set `sample` method was not balancing the sample by default, which is why I setup criteria for it. 

**The samples are already setup in the .csv files. You don't need to run this block of code (it won't even work) unless you upload the full .jsonl files.**

In [3]:
cat_comments = pd.read_json('data/categorized-comments.jsonl', lines=True)
cont_comments = pd.read_json('data/controversial-comments.jsonl', lines=True)

df1 = cont_comments[cont_comments['con']==1].sample(n=2500)
df0 = cont_comments[cont_comments['con']==0].sample(n=2500)
small_cont_comments = pd.concat([df1,df0])

small_cont_comments = cont_comments[cont_comments].sample(n=5000)
small_cat_comments = cat_comments.sample(n=5000)

small_cont_comments.to_csv(r'data/small_cont_comments.csv')
small_cat_comments.to_csv(r'data/small_cat_comments.csv')

print("NO NEED TO RUN THIS, JUST LOAD THE CSV(s) IN THE NEXT BLOCK!")

NO NEED TO RUN THIS, JUST LOAD THE CSV(s) IN THE NEXT BLOCK!


In order to avoid loading the entire data set each time, this code block independently loads the sample CSV files. This means that the previous step can be skipped every time except the first time (or any time you change the sample size).

This code should always work because at a minimum, the initial 1,000 observation files should be there (e.g., `small_cat_comments.csv`).

In [4]:
cat_comments = pd.read_csv(r'data/small_cat_comments.csv')
cont_comments = pd.read_csv(r'data/small_cont_comments.csv')

## Set the Target and Features

Within the categorical data set, the target is the `cat` (category) column and the feature (there's only one) is the `txt` column.

In [5]:
cat_target = cat_comments['cat']
cat_features = cat_comments['txt']
cont_target = cont_comments['con']
cont_features = cont_comments['txt']

# Show a little of the features and target
print("Categorical target sample:\n", cat_target[0:5], "\n")
print("Categorical feature sample:\n", cat_features[0:5], "\n")
print("Controversial target sample:\n", cont_target[0:5], "\n")
print("Controversial feature sample:\n", cont_features[0:5])

Categorical target sample:
 0               video_games
1    science_and_technology
2                    sports
3                    sports
4               video_games
Name: cat, dtype: object 

Categorical feature sample:
 0    I gotta say, Nintendo knocked it out of the pa...
1    What precious thing did you want to post that ...
2               Freeney sack@!\n\nGood job old timer!!
3    Don't blow it....keep it simple.... count your...
4    Which platform? When? Can't play till Friday p...
Name: txt, dtype: object 

Controversial target sample:
 0    1
1    1
2    1
3    1
4    1
Name: con, dtype: int64 

Controversial feature sample:
 0    Why is the a "bombshell?" If true and not anot...
1                               /s\n\nAt least I hope.
2    Well, theoretically the chances of democrats w...
3                         They always do unfortunately
4    After 2 years on it with crazy high deductible...
Name: txt, dtype: object


## Split to Train and Test Sets

The train set should be bigger since it's used to build the model, whereas the test set is used to validate the model with new data. For that reason, the test data is only 25% of the full set.

In [6]:
cat_features_train, cat_features_test, cat_target_train, cat_target_test = \
        train_test_split(cat_features, cat_target, test_size=0.25)

cont_features_train, cont_features_test, cont_target_train, cont_target_test = \
        train_test_split(cont_features, cont_target, test_size=0.25)

## Create the Model and Predictions

A pipeline is used and passed a TfidfVectorizer and MultinomailNB (Naive Bayes) objects. The pipeline, in turn, makes the words a vector of numbers so Naive Bayes can be used to predict the category.

In English, the sentences are broken down into words, which are the converted to numeric equivalents. Then, new sentences are broken down to numbers and compared to the numbers already created. When the algorithm finds the closest match, it chooses the category from that match as the prediction.

In [7]:
nb_model = make_pipeline(TfidfVectorizer(), MultinomialNB())
nb_model.fit(cat_features_train, cat_target_train)
nb_test_predictions = nb_model.predict(cat_features_test)

print("Small sample of the categorical predictions:", nb_test_predictions[0:5])

logistic_model = make_pipeline(TfidfVectorizer(), LogisticRegression())
logistic_model.fit(cont_features_train, cont_target_train)
logistic_test_predictions = logistic_model.predict(cont_features_test)

print("Small sample of the controversial predictions:", logistic_test_predictions[0:100])

Small sample of the categorical predictions: ['video_games' 'video_games' 'video_games' 'sports' 'sports']
Small sample of the controversial predictions: [1 1 1 0 0 1 0 0 1 0 1 0 0 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 1
 0 1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1 1 1 0 0 0 0 0
 0 0 0 1 0 0 0 1 0 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 0 1]


## Accuracy

Understanding how accurate the test is matters greatly and indicates how well a model will perform in the "real world." The lower the accuracy, the less likely the model will accurately predict a new sample. Unfortunately, the Naive Bayes model did not predict well, although that's likely for several reasons, including:

* The sample size is very small
* The text may require more tweaking (e.g., removing stop words and punctuation)
* Yeah, I think that's all...

In [8]:
cat_accuracy_test = accuracy_score(cat_target_test, nb_test_predictions)
print("The accuracy for the categorical test is:", cat_accuracy_test)

cont_accuracy_test = accuracy_score(cont_target_test, logistic_test_predictions)
print("The accuracy for the controversial test is:", cont_accuracy_test)

The accuracy for the categorical test is: 0.492
The accuracy for the controversial test is: 0.5576


## Out of Sample Predictions

Putting this in context, this model can be used as the 'engine' to make predictions based on new data. Taking a step back, in theory, the corpus and prediction can be anything you have the right data for - in this case, the data was great because every category comment had a category and every controversial commenet was noted as such. Without that level of detail, this engine would not be possible because you could not train a model as shown.

Here I'll take the same comment and run it through both the category and controvery models to see if works.

In [11]:
new_data_easy = "I love football!"
new_data_hard = "I'm a gamer for life!"

cat_pred = nb_model.predict([new_data_easy])
print("The easy predicted category is:", cat_pred)

cat_pred = nb_model.predict([new_data_hard])
print("The hard predicted category is:", cat_pred)

# The controversial test
new_data_nc = "Some vanilla statement, not controversial."
new_data_cont = "Abortion should be legal."

cont_pred = logistic_model.predict([new_data_nc])
print("The non-controversial comment prediction:", cont_pred)

cont_pred = logistic_model.predict([new_data_cont])
print("The controversial comment prediction:", cont_pred)

The easy predicted category is: ['sports']
The hard predicted category is: ['video_games']
The non-controversial comment prediction: [0]
The controversial comment prediction: [1]


## Observations

If this were a production application, any new sample that the system incorrectly predicted (e.g., txt="I really love this game. Football is amazing!" cat="sports") would be added to the corpus so next time the model was generated, it would work properly.